<a href="https://colab.research.google.com/github/ukamath/TransformersComparisons/blob/main/Transformers_Comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

The goal of this notebook is the create a simple example with [SimpleTransformers](https://https://simpletransformers.ai/) and text classification to illusrate the advantage of the API in performing comparisons and validations across different [HuggingFace](https://https://huggingface.co/) transformers.

#Environment Setup / Package Installation

In [50]:
!pip install simpletransformers


In [82]:
!pip install transformers-interpret

     |████████████████████████████████| 4.4MB 7.8MB/s 
  Created wheel for transformers-interpret: filename=transformers_interpret-0.3.0-cp37-none-any.whl size=13921 sha256=798e637349fe0e4b7ee51017412081eaea4335633578f636dc7ce3239022d468
  Stored in directory: /root/.cache/pip/wheels/b8/87/b5/75023c728155d0538a249cc15ece7174ca6450ea0bb5a66eec
Successfully built transformers-interpret


In [92]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

Sun Apr 18 18:09:37 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P0    35W / 250W |  16019MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

# Data Loading and Train/Valid/Test creation

This data originally came from Crowdflower's Data for Everyone library.https://www.figure-eight.com/data-for-everyone/

As the original source says,

A sentiment analysis job about the problems of each major U.S. airline. Twitter data was scraped from February of 2015 and contributors were asked to first classify positive, negative, and neutral tweets, followed by categorizing negative reasons (such as "late flight" or "rude service").

In [13]:
from google.colab import files
uploaded = files.upload()

Saving Tweets.csv to Tweets.csv


In [15]:
import numpy as np
# location of data
DATA_FILE = 'Tweets.csv'

# data columns for learning
TEXT_COLUMN_NAME = 'text'
LABEL_COLUMN_NAME = 'airline_sentiment'

# labels
labels = ['negative', 'neutral', 'positive']
tweets = pd.read_csv(DATA_FILE)
tweets = tweets.reindex(np.random.permutation(tweets.index))  
# only the text and labels
tweets = tweets[[TEXT_COLUMN_NAME, LABEL_COLUMN_NAME]]
# get a peek of the data
tweets.head()


,text,airline_sentiment
10703,@USAirways that's who I spoke to already. We'v...,neutral
4169,@united Thanks for the timely service &amp; gr...,positive
12781,"@AmericanAir the dinner and called me ""hon"". N...",neutral
12837,@AmericanAir this is a big company should be h...,negative
5402,@SouthwestAir Can I book a flight on my own an...,neutral


In [44]:
from sklearn.model_selection import train_test_split

X_train_entire, X_test, y_train_entire, y_test = train_test_split(
    tweets[TEXT_COLUMN_NAME], tweets[LABEL_COLUMN_NAME], test_size=0.15, random_state=37)
X_train, X_valid, y_train, y_valid = train_test_split(
    X_train_entire, y_train_entire, test_size=0.15, random_state=37)


In [45]:
from sklearn.preprocessing import LabelEncoder
# perform encoding of 
le = LabelEncoder()
y_train_le = le.fit_transform(y_train)
y_valid_le = le.fit_transform(y_valid)
y_test_le = le.transform(y_test)


In [46]:
training_data = {"text":X_train,"labels":y_train_le}
validation_data = {"text":X_valid,"labels":y_valid_le}
testing_data = {"text":X_test,"labels":y_test_le}


In [47]:
train_df = pd.DataFrame(training_data)
valid_df = pd.DataFrame(validation_data)
test_df = pd.DataFrame(testing_data)

In [61]:
def get_predictions(predictions):
  y_pred =[]
  confidence=[]
  for i in range(0, len(predictions)):
        predicted_index = np.argmax(predictions[i])
        y_pred.append(predicted_index)
        confidence.append(predictions[predicted_index])
  return y_pred, confidence


# Transformers and Classification

## BERT Model

In [48]:
from simpletransformers.classification import ClassificationModel
# Create a ClassificationModel
model = ClassificationModel(
    "bert", "bert-base-cased", num_labels=3, args={"use_cached_eval_features": True, "num_train_epochs": 5,"reprocess_input_data": True, "overwrite_output_dir": True, "best_model_dir": f"outputs/best_model", "save_eval_checkpoints": True}
)



Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [49]:
# Train the model
model.train_model(train_df, eval_df=valid_df)

  0%|          | 0/10577 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

(6615, 0.28051251447665854)

In [51]:
result, model_outputs, wrong_predictions = model.eval_model(test_df)

Running Evaluation:   0%|          | 0/275 [00:00<?, ?it/s]

In [63]:
from sklearn.metrics import classification_report
y_pred_bert, confidence_bert = get_predictions(model_outputs)
print(classification_report(y_test_le, y_pred_bert, target_names=labels))

              precision    recall  f1-score   support

    negative       0.88      0.92      0.90      1347
     neutral       0.70      0.66      0.68       452
    positive       0.84      0.78      0.81       397

    accuracy                           0.84      2196
   macro avg       0.81      0.78      0.79      2196
weighted avg       0.84      0.84      0.84      2196



## ROBERTA

In [56]:
# Create a ClassificationModel
roberta = ClassificationModel(
    "roberta", "roberta-base", num_labels=3, args={"use_cached_eval_features": True, "num_train_epochs": 5,"reprocess_input_data": True, "overwrite_output_dir": True, "best_model_dir": f"outputs/best_model", "save_eval_checkpoints": True}
)
# Train the model
roberta.train_model(train_df, eval_df=valid_df)

Some weights of the model checkpoint at roberta-base were not used when initializing RobertaForSequenceClassification: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at roberta-base and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out

  0%|          | 0/10577 [00:00<?, ?it/s]

Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Running Epoch 0 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 1 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

(6615, 0.353674940660403)

In [57]:
roberta_result, roberta_model_outputs, roberta_wrong_predictions = roberta.eval_model(test_df)

  0%|          | 0/2196 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/275 [00:00<?, ?it/s]

In [58]:
print(roberta_result)

{'mcc': 0.721409511750082, 'eval_loss': 0.8159207147427581}


In [64]:
from sklearn.metrics import classification_report
y_pred_roberta, confidence_roberta = get_predictions(roberta_model_outputs)
print(classification_report(y_test_le, y_pred_roberta, target_names=labels))

              precision    recall  f1-score   support

    negative       0.89      0.92      0.91      1347
     neutral       0.72      0.65      0.69       452
    positive       0.82      0.83      0.83       397

    accuracy                           0.85      2196
   macro avg       0.81      0.80      0.81      2196
weighted avg       0.85      0.85      0.85      2196



## DistilBERT

In [65]:
# Create a ClassificationModel
distilbert = ClassificationModel(
    "distilbert", "distilbert-base-cased", num_labels=3, args={"use_cached_eval_features": True, "num_train_epochs": 5,"reprocess_input_data": True, "overwrite_output_dir": True, "best_model_dir": f"outputs/best_model", "save_eval_checkpoints": True}
)
# Train the model
distilbert.train_model(train_df, eval_df=valid_df)

Downloading:   0%|          | 0.00/411 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/263M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-cased were not used when initializing DistilBertForSequenceClassification: ['vocab_transform.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-cased and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

...

Running Epoch 1 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

(6615, 0.2824988019281288)

In [66]:
distilbert_result, distilbert_model_outputs, distilbert_wrong_predictions = distilbert.eval_model(test_df)

  0%|          | 0/2196 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/275 [00:00<?, ?it/s]

In [67]:
from sklearn.metrics import classification_report
y_pred_distilbert, confidence_distilbert = get_predictions(distilbert_model_outputs)
print(classification_report(y_test_le, y_pred_distilbert, target_names=labels))

              precision    recall  f1-score   support

    negative       0.88      0.92      0.90      1347
     neutral       0.69      0.63      0.66       452
    positive       0.82      0.77      0.79       397

    accuracy                           0.83      2196
   macro avg       0.79      0.77      0.78      2196
weighted avg       0.83      0.83      0.83      2196



## ELECTRA

In [69]:
# Create a ClassificationModel
electra = ClassificationModel(
    "electra", "google/electra-base-discriminator", num_labels=3, args={"use_cached_eval_features": True, "num_train_epochs": 5,"reprocess_input_data": True, "overwrite_output_dir": True, "best_model_dir": f"outputs/best_model", "save_eval_checkpoints": True}
)
# Train the model
electra.train_model(train_df, eval_df=valid_df)

Downloading:   0%|          | 0.00/467 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at google/electra-base-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.weight', 'discriminator_predictions.dense.bias', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at google/electra-base-discriminator and are newly initialized: ['pooler.dense

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Running Epoch 1 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

(6615, 0.352653571008687)

In [70]:
electra_result, electra_model_outputs, electra_wrong_predictions = electra.eval_model(test_df)

  0%|          | 0/2196 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/275 [00:00<?, ?it/s]

In [71]:
from sklearn.metrics import classification_report
y_pred_electra, confidence_electra = get_predictions(electra_model_outputs)
print(classification_report(y_test_le, y_pred_electra, target_names=labels))

              precision    recall  f1-score   support

    negative       0.89      0.90      0.89      1347
     neutral       0.65      0.65      0.65       452
    positive       0.79      0.77      0.78       397

    accuracy                           0.82      2196
   macro avg       0.78      0.77      0.78      2196
weighted avg       0.82      0.82      0.82      2196



## XLNet

In [72]:
# Create a ClassificationModel
xlnet = ClassificationModel(
    "xlnet", "xlnet-base-cased", num_labels=3, args={"use_cached_eval_features": True, "num_train_epochs": 5,"reprocess_input_data": True, "overwrite_output_dir": True, "best_model_dir": f"outputs/best_model", "save_eval_checkpoints": True}
)
# Train the model
xlnet.train_model(train_df, eval_df=valid_df)

Downloading:   0%|          | 0.00/760 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/467M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlnet-base-cased were not used when initializing XLNetForSequenceClassification: ['lm_loss.weight', 'lm_loss.bias']
- This IS expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLNetForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLNetForSequenceClassification were not initialized from the model checkpoint at xlnet-base-cased and are newly initialized: ['sequence_summary.summary.weight', 'sequence_summary.summary.bias', 'logits_proj.weight', 'logits_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions a

Downloading:   0%|          | 0.00/798k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.38M [00:00<?, ?B/s]

...

Running Epoch 1 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

(6615, 0.34140421195034987)

In [73]:
xlnet_result, xlnet_model_outputs, xlnet_wrong_predictions = xlnet.eval_model(test_df)

  0%|          | 0/2196 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/275 [00:00<?, ?it/s]

In [74]:
from sklearn.metrics import classification_report
y_pred_xlnet, confidence_xlnet = get_predictions(xlnet_model_outputs)
print(classification_report(y_test_le, y_pred_xlnet, target_names=labels))

              precision    recall  f1-score   support

    negative       0.88      0.92      0.90      1347
     neutral       0.72      0.60      0.65       452
    positive       0.81      0.82      0.82       397

    accuracy                           0.84      2196
   macro avg       0.80      0.78      0.79      2196
weighted avg       0.83      0.84      0.83      2196



## XLM

In [77]:
# Create a ClassificationModel
xlm = ClassificationModel(
    "xlm", "xlm-clm-enfr-1024", num_labels=3, args={"use_cached_eval_features": True, "num_train_epochs": 5,"reprocess_input_data": True, "overwrite_output_dir": True, "best_model_dir": f"outputs/best_model", "save_eval_checkpoints": True}
)
# Train the model
xlm.train_model(train_df, eval_df=valid_df)

Downloading:   0%|          | 0.00/1.02k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/830M [00:00<?, ?B/s]

Some weights of the model checkpoint at xlm-clm-enfr-1024 were not used when initializing XLMForSequenceClassification: ['pred_layer.proj.weight', 'pred_layer.proj.bias']
- This IS expected if you are initializing XLMForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing XLMForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of XLMForSequenceClassification were not initialized from the model checkpoint at xlm-clm-enfr-1024 and are newly initialized: ['transformer.position_ids', 'sequence_summary.summary.weight', 'sequence_summary.summary.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and i

Downloading:   0%|          | 0.00/1.45M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.01M [00:00<?, ?B/s]

...

Running Epoch 1 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 2 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 3 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

Running Epoch 4 of 5:   0%|          | 0/1323 [00:00<?, ?it/s]

(6615, 0.7018638955179999)

In [78]:
xlm_result, xlm_model_outputs, xlm_wrong_predictions = xlm.eval_model(test_df)

  0%|          | 0/2196 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/275 [00:00<?, ?it/s]

In [79]:
from sklearn.metrics import classification_report
y_pred_xlm, confidence_xlm = get_predictions(xlm_model_outputs)
print(classification_report(y_test_le, y_pred_xlm, target_names=labels))

              precision    recall  f1-score   support

    negative       0.85      0.90      0.87      1347
     neutral       0.60      0.54      0.57       452
    positive       0.73      0.67      0.70       397

    accuracy                           0.78      2196
   macro avg       0.73      0.70      0.71      2196
weighted avg       0.78      0.78      0.78      2196



# Interpretability and Transformers

In [93]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
model_name = "bert-base-cased"
tokenizer =  model.tokenizer



In [94]:
from transformers_interpret import SequenceClassificationExplainer
# bert model transformers
cls_explainer = SequenceClassificationExplainer(
    model.model, 
    tokenizer)
word_attributions = cls_explainer(" I love flying on delta")

In [95]:
cls_explainer.visualize("bert_viz.html")

True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (1.00),LABEL_2,1.29,[CLS] I love flying on delta [SEP]


True Label,Predicted Label,Attribution Label,Attribution Score,Word Importance
2,LABEL_2 (1.00),LABEL_2,1.29,[CLS] I love flying on delta [SEP]
